In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fine Tuning T5 1.1 XL on WMT task

In [3]:
import os
import time
from datetime import datetime
import pandas as pd

from google.cloud import aiplatform as vertex_ai

## Configure environment settings

In [14]:
PROJECT_ID = 'jk-mlops-dev' # Change to your project id.
REGION = 'us-central1'  # Change to your region.
BUCKET = 'jk-t5x-staging' # Change to your bucket.
TENSORBOARD_NAME = 't5x-experiments' # Change to your bucket.
TENSORBOARD_ID = ! gcloud ai tensorboards list --filter="displayName={TENSORBOARD_NAME}" --format="value(name)" --region=us-central1 
TENSORBOARD_ID = TENSORBOARD_ID[1]
IMAGE_NAME = 't5x-base'
IMAGE_URI = f'gcr.io/{PROJECT_ID}/{IMAGE_NAME}'

## Configure experiment settings

In [35]:
EXPERIMENT_NAME = 'fine-tune-t5-xl'

EXPERIMENT_WORKSPACE = f'gs://{BUCKET}/experiments/{EXPERIMENT_NAME}'
TFDS_DATA_DIR = f'{EXPERIMENT_WORKSPACE}/dataset'
EXPERIMENT_RUNS = f'{EXPERIMENT_WORKSPACE}/runs'

In [36]:
vertex_ai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=EXPERIMENT_WORKSPACE,
    experiment=EXPERIMENT_NAME
)

## Configure a fine tuning job

### Create a gin file

In [37]:
gin_file_name = 'finetune_t5_xl_wmt.gin'

In [58]:
%%writefile {gin_file_name}
from __gin__ import dynamic_registration

import __main__ as train_script
from t5.data import mixtures
from t5x import models
from t5x import partitioning
from t5x import utils

include "t5x/examples/t5/t5_1_1/xl.gin"
include "t5x/configs/runs/finetune.gin"

MIXTURE_OR_TASK_NAME = "wmt_t2t_ende_v003"
TASK_FEATURE_LENGTHS = {"inputs": 256, "targets": 256}
TRAIN_STEPS = 1_020_000  # 1000000 pre-trained steps + 20000 fine-tuning steps.
DROPOUT_RATE = 0.0
INITIAL_CHECKPOINT_PATH = "gs://t5-data/pretrained_models/t5x/t5_1_1_xl/checkpoint_1000000"
LOSS_NORMALIZING_FACTOR = 233472

BATCH_SIZE = 64
partitioning.PjitPartitioner.num_partitions = 8

Overwriting finetune_t5_xl_wmt.gin


### Configure run artifact repo and copy gin configuration

*We may want to encapsulate the following steps in a utility function, as this is a lot of boilerplate code. For now I removed experiment logging code as we need to rethink and simplify how we do it*

In [59]:
RUN_NAME = f'run_{datetime.now().strftime("%Y%m%d%H%M")}'
RUN_DIR = f'{EXPERIMENT_RUNS}/{RUN_NAME}'
! gsutil cp {gin_file_name} {RUN_DIR}/
GIN_FILE_GCS = f'{RUN_DIR.replace("gs://", "/gcs/")}/{gin_file_name}'

Copying file://finetune_t5_xl_wmt.gin [Content-Type=application/octet-stream]...
/ [1 files][  646.0 B/  646.0 B]                                                
Operation completed over 1 objects/646.0 B.                                      


### Configure Vertex AI CustomJob

In [60]:
RUN_MODE = 'train'

MACHINE_TYPE = 'cloud-tpu'
ACCELERATOR_TYPE = 'TPU_V2'
ACCELERATOR_NUM = 32
REPLICA_COUNT = 1

worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": MACHINE_TYPE,
            "accelerator_type": ACCELERATOR_TYPE,
            "accelerator_count": ACCELERATOR_NUM,
        },
        "replica_count": REPLICA_COUNT,
        "container_spec": {
            "image_uri": IMAGE_URI,
            "args": [
                f'--run_mode={RUN_MODE}',
                f'--gin_file={GIN_FILE_GCS}',
                f'--gin.MODEL_DIR="{RUN_DIR}"',
                f'--tfds_data_dir={TFDS_DATA_DIR}',
                '--gin.USE_CACHED_TASKS=False'
            ],
        },
    }
]

worker_pool_specs

[{'machine_spec': {'machine_type': 'cloud-tpu',
   'accelerator_type': 'TPU_V2',
   'accelerator_count': 32},
  'replica_count': 1,
  'container_spec': {'image_uri': 'gcr.io/jk-mlops-dev/t5x-base',
   'args': ['--run_mode=train',
    '--gin_file=/gcs/jk-t5x-staging/experiments/fine-tune-t5-xl/runs/run_202207231737/finetune_t5_xl_wmt.gin',
    '--gin.MODEL_DIR="gs://jk-t5x-staging/experiments/fine-tune-t5-xl/runs/run_202207231737"',
    '--tfds_data_dir=gs://jk-t5x-staging/experiments/fine-tune-t5-xl/dataset',
    '--gin.USE_CACHED_TASKS=False']}}]

### Submit the job

In [61]:
job = vertex_ai.CustomJob(
    display_name=RUN_NAME,
    worker_pool_specs=worker_pool_specs,
    base_output_dir=RUN_DIR
)
job.run(
    sync=False
)

Creating CustomJob


INFO:google.cloud.aiplatform.jobs:Creating CustomJob


CustomJob created. Resource name: projects/895222332033/locations/us-central1/customJobs/1712940434906415104


INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/895222332033/locations/us-central1/customJobs/1712940434906415104


To use this CustomJob in another session:


...

INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/1712940434906415104 current state:
JobState.JOB_STATE_RUNNING


CustomJob projects/895222332033/locations/us-central1/customJobs/1712940434906415104 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/1712940434906415104 current state:
JobState.JOB_STATE_RUNNING


CustomJob projects/895222332033/locations/us-central1/customJobs/1712940434906415104 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/1712940434906415104 current state:
JobState.JOB_STATE_RUNNING


### Monitor the job with Vertex Tensorboard

*During the Vertex AI Training Preview, native integration with Vertex Tensorboard is not supported. As a mitigation you can use tb-gcp-uploader to manually start tensorboard logs uploading*

*Execute the following command from the terminal window*

In [52]:
cmd = f"""
tb-gcp-uploader --tensorboard_resource_name {TENSORBOARD_ID} \
--logdir {EXPERIMENT_RUNS} \
--experiment_name {EXPERIMENT_NAME}
"""

print(cmd)


tb-gcp-uploader --tensorboard_resource_name projects/895222332033/locations/us-central1/tensorboards/2937103421045473280 --logdir gs://jk-t5x-staging/experiments/fine-tune-t5-xl/runs --experiment_name fine-tune-t5-xl

CustomJob projects/895222332033/locations/us-central1/customJobs/1320001367418339328 current state:
JobState.JOB_STATE_PENDING


INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/1320001367418339328 current state:
JobState.JOB_STATE_PENDING


CustomJob projects/895222332033/locations/us-central1/customJobs/1320001367418339328 current state:
JobState.JOB_STATE_PENDING


INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/1320001367418339328 current state:
JobState.JOB_STATE_PENDING


CustomJob projects/895222332033/locations/us-central1/customJobs/1320001367418339328 current state:
JobState.JOB_STATE_PENDING


...

INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/1320001367418339328 current state:
JobState.JOB_STATE_RUNNING


CustomJob projects/895222332033/locations/us-central1/customJobs/1320001367418339328 current state:
JobState.JOB_STATE_RUNNING


INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/1320001367418339328 current state:
JobState.JOB_STATE_RUNNING


CustomJob projects/895222332033/locations/us-central1/customJobs/1320001367418339328 current state:
JobState.JOB_STATE_FAILED


INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/1320001367418339328 current state:
JobState.JOB_STATE_FAILED


In [ ]:
vertex_ai.end_run()